In [18]:
import os
import copy
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io
import matplotlib.pyplot as plt
import glob
import re
from datetime import datetime
from dateutil import parser
import dateutil.parser
from scipy.io import savemat

import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import math
from matplotlib import pyplot as plt
import statistics
from sklearn.ensemble import IsolationForest

In [2]:
FREESURFER_A_USAR = 'freesurfer_cross_long_a_usar.csv'
ASEG_STATS = 'aseg_stats.csv'
LIPIDOMICS = 'lipidomics_v3.csv'
NAO_PROCESSADOS = 'nao_processados_a_usar_melhores_expandido.csv'
DATASET = 'DatasetPro4.0.csv'
MMSE = 'MMSE.csv'

In [5]:
volumes= pd.read_csv('rh_aparc.DKTatlas_stats.csv')
# lipidomics = pd.read_csv('D:\Tese\lipidomics_v3.csv')

rid = []
viscode = []
for index, row in volumes.iterrows():
    match = re.search(r"S_0?(.*?)\.", row['rh.aparc.DKTatlas.volume']).group(1)
    match2 = re.search(r"\.(.*?)$", row['rh.aparc.DKTatlas.volume']).group(1)
    rid.append(match)
    viscode.append(match2)
volumes.pop('rh.aparc.DKTatlas.volume')
volumes.insert(0, 'RID', rid)
volumes.insert(1, 'VISCODE', viscode)
volumes.to_csv('rh_aparc.DKTatlas_stats1.0.csv', index=False)

In [ ]:
subject_list= pd.read_csv('D:\Tese\SubjectList.csv')
dataset11= pd.read_csv('D:\Tese\Dataset_Final\Dataset1.1.csv')

age = []
sex = []
date = []
group = []
for index1, row1 in dataset11.iterrows():
    for index2, row2 in subject_list.iterrows():
        if (row1['RID'] == row2['RID']) and (row1['VISCODE'] == row2['VISCODE']):
            age.append(row2['Age'])
            sex.append(row2['Sex'])
            date.append(row2['Acq Date'])
            group.append(row2['Group'])
dataset11.insert(2, 'Age', age)
dataset11.insert(3, 'Sex', sex)
dataset11.insert(4, 'MRI_Date', date)
dataset11.insert(5, 'Group', group)
dataset11.to_csv('Dataset1.2.csv', index=False)

In [ ]:
dataset12= pd.read_csv('D:\Tese\Dataset_Final\Dataset1.2.csv')
lipidomics = pd.read_csv('D:\Tese\lipidomics_v3.csv')

dataset13 = pd.merge(dataset12, lipidomics, on=['RID', 'VISCODE'], how='inner')
dataset13 = dataset13.drop_duplicates(subset=['RID', 'VISCODE'])
lipidomics_date = dataset13.pop('EXAMDATE')
viscode2 = dataset13.pop('VISCODE2')
dataset13.insert(6, 'EXAMDATE', lipidomics_date)
dataset13.insert(7, 'VISCODE2', viscode2)
dataset13 = dataset13.sort_values(by=['RID', 'MRI_Date'], ascending=[True, True])
dataset13.to_csv('Dataset1.3.csv', index=False)

In [6]:
merged = pd.read_csv('merged.csv')
merged = merged.sort_values(by=['RID', 'Exam_Date'], ascending=[True, True])
merged.to_csv('merged_final.csv', index=False)

In [11]:
dataset= pd.read_csv('DatasetPro6.0.csv')
rh= pd.read_csv('rh_aparc.DKTatlas_stats.csv')
lh= pd.read_csv('lh_aparc.DKTatlas_stats.csv')
merged_aparc = pd.merge(rh, lh, on=['RID', 'VISCODE'], how='inner')
merged_aparc = merged_aparc.drop(columns=['BrainSegVolNotVent_x', 'eTIV_x', 'BrainSegVolNotVent_y', 'eTIV_y'])
merged_df = pd.merge(dataset, merged_aparc, on=['RID', 'VISCODE'], how='inner')
merged_df.to_csv('DatasetPro7.0.csv', index=False)

In [16]:
dataset= pd.read_csv('DatasetPro7.0.csv')
count = 0
for col in dataset.columns[1639:1704]:
    term = dataset.pop(col)
    dataset.insert(9+count, col, term)
    count +=1
dataset.to_csv('DatasetPro8.0.csv', index=False)

### Total Intracranial Volume Ratio

In [ ]:
dataset = pd.read_csv(DATASET)
dataset['LHC_ICV'] = None
dataset['RHC_ICV'] = None
for index, row in dataset.iterrows():
        l = round(row['Left-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        r = round(row['Right-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        dataset.at[index, 'LHC_ICV'] = l
        dataset.at[index, 'RHC_ICV'] = r
        print(dataset.loc[index, 'RID'])

dataset.to_csv('DatasetPro2.0.csv', index=False)

### Add MMSE Values to Dataset

In [ ]:
dataset = pd.read_csv('DatasetPro2.0.csv')
mmse = pd.read_csv(MMSE)
dataset['MMSE'] = None
for index, row in dataset.iterrows():
    for index2, row2 in mmse.iterrows():
        if (row['RID'] == row2['RID']) and (row['VISCODE2'] == row2['VISCODE2']):
            dataset.at[index, 'MMSE'] = row2['MMSCORE']
            print(dataset.loc[index, 'RID'])
dataset.to_csv('DatasetPro3.0.csv', index=False)


### Normalizacao de Volumes

In [ ]:
dataset = pd.read_csv('DatasetPro8.0.csv')

for index, row in dataset.iterrows():
    for i in dataset.columns[9:132]:
        value = round(row[i]/row['EstimatedTotalIntraCranialVol'],9)
        new_col = i+'_ICV'
        dataset.at[index, new_col] = value
dataset.to_csv('DatasetPro9.0.csv', index=False)  


# print(dataset.columns[9:132])

### Normalizacao de Lipidos

In [8]:
DATASET = 'DatasetPro3.0.csv'

In [ ]:
dataset = pd.read_csv(DATASET)

for index, row in dataset.iterrows():
    total = row[dataset.columns[74:855]].sum()
    for i in dataset.columns[74:855]:
        new_col = i+'%mol'
        perc_mol = (row[i]/total) * 100
        dataset.at[index, new_col] = perc_mol
dataset.to_csv('DatasetPro4.0.csv', index=False)    

### MMSE Missing Values Handling

In [27]:
DATASET = 'DatasetPro5.0.csv'
dataset = pd.read_csv(DATASET)

group_items = []
for index, row in dataset.iterrows():
    if row['Group'] not in group_items:
        group_items.append(row['Group'])
print(group_items)

#---------------------------------------------------------------------------------------------------------------
valuesAD = dataset.loc[dataset['Group'] == 'AD', 'MMSE'].tolist()
ad = [x for x in valuesAD if not np.isnan(x)]
median_value_ad = statistics.median(ad)

valuesMCI = dataset.loc[dataset['Group'] == 'MCI', 'MMSE'].tolist()
mci = [x for x in valuesMCI if not np.isnan(x)]
median_value_mci = statistics.median(mci)

valuesCN = dataset.loc[dataset['Group'] == 'CN', 'MMSE'].tolist()
cn = [x for x in valuesCN if not np.isnan(x)]
median_value_cn = statistics.median(cn)

valuesEMCI = dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'].tolist()
emci = [x for x in valuesEMCI if not np.isnan(x)]
median_value_emci = statistics.median(emci)

valuesLMCI = dataset.loc[dataset['Group'] == 'LMCI', 'MMSE'].tolist()
lmci = [x for x in valuesLMCI if not np.isnan(x)]
median_value_lmci = statistics.median(lmci)

valuesSMC = dataset.loc[dataset['Group'] == 'SMC', 'MMSE'].tolist()
smc = [x for x in valuesSMC if not np.isnan(x)]
median_value_smc = statistics.median(smc)

print(median_value_ad,median_value_mci,median_value_cn,median_value_emci,median_value_lmci,median_value_smc)

#-----------------------------------------------------------------------------------------------------------

dataset.loc[dataset['Group'] == 'AD', 'MMSE'] = dataset.loc[dataset['Group'] == 'AD', 'MMSE'].fillna(median_value_ad)
dataset.loc[dataset['Group'] == 'MCI', 'MMSE'] = dataset.loc[dataset['Group'] == 'MCI', 'MMSE'].fillna(median_value_mci)
dataset.loc[dataset['Group'] == 'CN', 'MMSE'] = dataset.loc[dataset['Group'] == 'CN', 'MMSE'].fillna(median_value_cn)
dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'] = dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'].fillna(median_value_emci)
dataset.loc[dataset['Group'] == 'lmci', 'MMSE'] = dataset.loc[dataset['Group'] == 'lmci', 'MMSE'].fillna(median_value_lmci)
dataset.loc[dataset['Group'] == 'SMC', 'MMSE'] = dataset.loc[dataset['Group'] == 'SMC', 'MMSE'].fillna(median_value_smc)
dataset.to_csv('DatasetPro6.0.csv', index=False)  

['AD', 'MCI', 'CN', 'EMCI', 'LMCI', 'SMC']
22.0 26.0 29.0 29.0 26.0 29.0


### Creating Lipid Class Columns

In [3]:
DATASET = 'DatasetPro9.0.csv'
dataset = pd.read_csv(DATASET)
features = []
for feature in dataset.columns[923:1703]:
    match = re.search(r'^[^.]+', feature)
    i = match.group(0)
    if i not in features:
        features.append(i)

swap_dict = {
    'CA%mol': 'CA',
    'DXCA%mol': 'DXCA',
    'COH%mol': 'COH',
    'UBIQUINONE%mol': 'UBIQUINONE'
}
features = [swap_dict.get(item, item) for item in features]
print(features)
 
dataset.drop(columns=dataset.columns[141:922], inplace=True)

for feature in features:
    feature_sum = []
    matched_columns = [col for col in dataset.columns[141:922] if re.search(feature, col)]
    if matched_columns:
        feature_sum = dataset[matched_columns].sum(axis=1)
    else:
        feature_sum = [0] * len(dataset)
    dataset[feature] = feature_sum

dataset.to_csv('DatasetPro10.0.csv', index=False)

Index(['SPH.D18.1.%mol', 'SPH.D18.2.%mol', 'S1P.D16.1.%mol', 'S1P.D18.0.%mol',
       'S1P.D18.1.%mol', 'S1P.D18.2.%mol', 'DHCER.D18.0.16.0.%mol',
       'DHCER.D18.0.18.0.%mol', 'DHCER.D18.0.20.0.%mol',
       'DHCER.D18.0.22.0.%mol',
       ...
       'TG.O.54.4...NL.18.2.%mol', 'UBIQUINONE%mol', 'CE.18.2....OH.%mol',
       'CE.20.4....OH.%mol', 'CE.22.6....OH.%mol', 'LPC.18.2....OH.%mol',
       'LPC.20.4....OH.%mol', 'LPC.22.6....OH.%mol', 'PC.34.2....OH.%mol',
       'PC.36.4....OH.%mol'],
      dtype='object', length=781)


## Outlier Detection

### BoxPlot

In [17]:
DATASET = "Statistics/test/TempDatasetPro10.csv"
dataset = pd.read_csv(DATASET)
# filtered_rows = dataset[(dataset['Age'] > 70) & (dataset['Group'] == "MCI")]
df_filtered = dataset.drop(dataset[(dataset['Age'] > 70) & (dataset['Group'] == "MCI")].index)

def outliers(df,ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    ls = df.index[(df[ft] < lower_bound) | (df[ft] > upper_bound)]

    return ls

index_list = []
for feature in df_filtered.columns[1046:1080]:
    index_list.extend(outliers(df_filtered, feature))
print(index_list)


[8, 29, 287, 636, 685, 710, 752, 965, 1229, 1230, 1339, 1547, 1830, 2077, 2102, 2658, 2713, 22, 24, 25, 28, 114, 227, 228, 606, 608, 609, 610, 654, 655, 843, 1237, 1239, 1502, 1545, 1546, 1837, 1875, 1876, 1881, 1927, 1942, 1951, 1952, 2041, 2042, 2086, 2094, 2113, 2115, 2138, 2159, 2160, 2161, 2166, 2189, 2197, 2204, 2207, 2208, 2225, 2227, 2255, 2296, 2321, 2322, 2331, 2375, 2401, 2431, 2439, 2536, 2602, 2603, 2612, 2634, 2684, 2726, 2739, 21, 22, 26, 542, 936, 1247, 1400, 1407, 1821, 1866, 1875, 1876, 2038, 2039, 2059, 2060, 2086, 2138, 2189, 2197, 2214, 2255, 2290, 2568, 2572, 2576, 2593, 2603, 2627, 2659, 2691, 5, 8, 64, 104, 112, 113, 153, 170, 171, 172, 201, 214, 249, 412, 470, 641, 649, 704, 706, 733, 897, 902, 936, 937, 941, 1730, 1765, 1781, 1782, 1783, 1805, 1816, 1817, 2063, 2064, 2083, 2315, 2340, 2481, 2538, 757, 848, 849, 850, 851, 928, 929, 932, 1137, 1267, 1540, 1547, 1548, 1715, 1716, 1764, 1766, 1891, 1951, 1952, 1965, 1994, 2037, 2080, 2128, 2141, 2244, 2245, 2291, 

### Isolation Forest

In [25]:
DATASET = "Statistics/test/TempDatasetPro10.csv"
dataset = pd.read_csv(DATASET)
# filtered_rows = dataset[(dataset['Age'] > 70) & (dataset['Group'] == "MCI")]
df_filtered = dataset.drop(dataset[(dataset['Age'] > 70) & (dataset['Group'] == "MCI")].index)
df_filtered = df_filtered.iloc[:, 1045:1080]

iso_forest = IsolationForest(contamination=0.01, random_state=42)

# Fit the model and predict outliers
outliers = iso_forest.fit_predict(df_filtered)

# Mark outliers
df_filtered['outlier'] = outliers

# Print the rows classified as outliers
print(df_filtered[df_filtered['outlier'] == -1])

           SPH       S1P     DHCER       CER     CER1P    HEXCER   HEX2CER  \
494   0.003019  0.007524  0.006396  0.195131  0.000067  0.016439  0.011915   
606   0.001747  0.014221  0.021064  0.243990  0.000048  0.021507  0.010307   
618   0.000469  0.008508  0.010436  0.317340  0.000030  0.013024  0.012117   
872   0.000770  0.016696  0.006526  0.320566  0.000128  0.050735  0.031469   
902   0.004640  0.009545  0.007609  0.266929  0.000217  0.045912  0.031250   
1416  0.000585  0.008275  0.009276  0.294153  0.000036  0.015181  0.012266   
1765  0.001010  0.018560  0.008006  0.369854  0.000211  0.054222  0.037921   
1771  0.000396  0.008010  0.008535  0.232823  0.000051  0.013849  0.008396   
1805  0.000921  0.018137  0.006261  0.279382  0.000222  0.037285  0.030451   
1865  0.001430  0.017147  0.011003  0.358471  0.000072  0.028518  0.011824   
1999  0.003392  0.023957  0.008763  0.229067  0.000057  0.024230  0.012975   
2086  0.001664  0.021193  0.013356  0.420266  0.000047  0.028935

c:\Users\ciara\anaconda3\envs\Monai\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
